In [ ]:
from sklearn import metrics

def getNeighborMedian(nbr):
    if nbr == 'MeadowV':
        return 88
    elif nbr == 'IDOTRR':
        return 103
    elif nbr == 'BrDale':
        return 106  
    elif nbr == 'OldTown':
        return 119   
    elif nbr == 'Edwards':
        return 122
    elif nbr == 'BrkSide':
        return 124
    elif nbr == 'Sawyer':
        return 135
    elif nbr == 'Blueste':
        return 138
    elif nbr == 'SWISU':
        return 140
    elif nbr == 'NAmes':
        return 140
    elif nbr == 'NPkVill':
        return 146
    elif nbr == 'Mitchel':
        return 154
    elif nbr == 'SawyerW':
        return 182    
    elif nbr == 'Gilbert':
        return 182   
    elif nbr == 'NWAmes':
        return 182    
    elif nbr == 'Blmngtn':
        return 191   
    elif nbr == 'CollgCr':
        return 197
    elif nbr == 'ClearCr':
        return 201
    elif nbr == 'Crawfor':
        return 201
    elif nbr == 'Veenker':
        return 218
    elif nbr == 'Somerst':
        return 223
    elif nbr == 'Timber':
        return 228
    elif nbr == 'StoneBr':
        return 278  
    elif nbr == 'NoRidge':
        return 302   
    elif nbr == 'NridgHt':
        return 315  
    
def rmsle(y_true,y_pred):
   assert len(y_true) == len(y_pred)
   return np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean() ** 0.5
    

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

%matplotlib inline
#panda is a read/write tool

train_file = "c:\\house\\train.csv"
test_file = "c:\\house\\test.csv"

# comma delimited is the default
hsData = pd.read_csv(train_file, header = 0)
hsData = hsData.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'LotFrontage', 'FireplaceQu', 'Utilities'], axis=1)

# The 'Sold_Time' is usefless
#hsData['Sold_Time'] = (hsData['YrSold'] - 2005)*12 + hsData['MoSold']

# The composite 'Total_FlrSF' improves some
hsData['Total_FlrSF'] = hsData['1stFlrSF'] + hsData['2ndFlrSF']

# The composite 'Total_Bath' improves a bit
hsData['Total_Bath'] = hsData['FullBath'] + hsData['BsmtFullBath'] + hsData['HalfBath']*0.5 + hsData['BsmtHalfBath']*0.5

# The composite 'Garage_Composite', 'Exter_Composite' and 'Bsmt_Composite' improves a lot
hsData['GarageQual'] = hsData['GarageQual'].fillna('NA')
hsData['GarageCond'] = hsData['GarageCond'].fillna('NA')
hsData['Garage_Composite'] = hsData['GarageQual'] + hsData['GarageCond']
hsData['Garage_Composite'].replace(['PoPo','NANA','FaFa','FaPo','FaTA','ExEx','TAFa','TAGd', \
                             'TATA','GdTA','GdGd','ExTA'], \
                             [97,100,110,116,116,125,125,144,171,209,227,475], inplace=True)

hsData['Exter_Composite'] = hsData['ExterQual'] + hsData['ExterCond']
hsData['Exter_Composite'].replace(['FaFa','TAPo','FaTA','TAFa','TAEx','TATA','TAGd','GdGd', \
                             'GdTA','ExGd','ExEx','ExTA'], \
                             [61,77,99,119,140,140,140,210,221,267,325,374], inplace=True)

hsData['BsmtQual'] = hsData['BsmtQual'].fillna('NA')
hsData['BsmtCond'] = hsData['BsmtCond'].fillna('NA')
hsData['Bsmt_Composite'] = hsData['BsmtQual'] + hsData['BsmtCond']
hsData['Bsmt_Composite'].replace(['FaPo','NANA','FaTA','FaFa','TAFa','TATA','TAGd','GdFa', \
                            'GdTA','GdGd','ExGd','ExTA'], \
                             [64,112,114,114,114,135,164,182,192,199,275,318], inplace=True)

hsData['Year_Composite'] = hsData['YearBuilt'] + hsData['YearRemodAdd']
hsData['Year_Composite'] = hsData['Year_Composite'].apply(lambda year: year - 3700)

hsData['HeatingQC'].replace(['Po','Fa','TA','Gd','Ex'], [87,124,135,152,195], inplace=True)
hsData['KitchenQual'].replace(['NA', 'Fa', 'TA', 'Gd', 'Ex'], [105565,115000,137000,201400,316750], inplace=True)

hsData['Condition1'].replace(['Artery', 'Feedr', 'RRAe', 'Norm', 'RRAn', 'RRNe', 'PosN', 'PosA', 'RRNn'], [120,140,143,167,171,191,200,212,214], inplace=True)
hsData['Condition2'].replace(['RRNn', 'Artery', 'Feedr', 'RRAn', 'Norm', 'RRAe', 'PosN', 'PosA'], [97,107,128,137,164,190,285,325], inplace=True)

hsData['CentralAir'] = hsData['CentralAir'].apply(lambda air: 1 if air == 'Y' else 0)

hsData['Electrical'] = hsData['Electrical'].fillna('Mix')
hsData['Electrical'].replace(['Mix', 'FuseP', 'FuseF', 'FuseA', 'SBrkr'], [67,82,115,121,170], inplace=True)

hsData['Functional'].replace(['Maj2','Sev','Mod','Min1','Min2','Maj1','Typ'], [85,129,140,140,140,140,166], inplace=True)

hsData['SaleType'].replace(['Oth','ConLI','COD','ConLD','ConLw','WD','CWD','New','Con'], [116,125,140,140,144,158,189,247,270], inplace=True)

hsData['SaleCondition'].replace(['AdjLand', 'Abnorml', 'Family', 'Alloca', 'Normal', 'Partial'], [104,130,141,148,160,245], inplace=True)

hsData['GarageType'] = hsData['GarageType'].fillna('Basment')
hsData['GarageType'].replace(['CarPort','Detchd','Basment','2Types','Attchd','BuiltIn'], \
                             [108,130,148,159,185,228], inplace=True)

hsData['MSZoning'].replace(['C (all)','RM','RH','RL','FV'], [75,121,137,174,206], inplace=True)

hsData['MSSubClass'] = hsData['MSSubClass'].apply(str)
hsData['MSSubClass'].replace(['180','30','45','190','50','90','85','40','160','70','20','75','80','120','60'], \
                             [89,100,108,128,132,136,141,143,146,156,159,164,167,192,215], inplace=True)

hsData['BldgType'].replace(['2fmCon','Duplex','Twnhs','1Fam','TwnhsE'], [128,137,137,168,172], inplace=True)

hsData['HouseStyle'].replace(['1.5Unf','1.5Fin','2.5Unf','SFoyer','1Story','SLvl','2Story','2.5Fin'], [111,132,135,135,155,165,190,194], inplace=True)

hsData['LotShape'].replace(['Reg','IR1','IR2','IR3'], [146,189,204,221], inplace=True)

hsData['LandContour'].replace(['Lvl','Bnk','Low','HLS'], [139,163,190,222], inplace=True)

hsData['GarageFinish'] = hsData['GarageFinish'].fillna('Unf')
hsData['GarageFinish'].replace(['Unf','RFn','Fin'], [135,190,215], inplace=True)

print hsData['GarageFinish'].unique()
# Trim below noise improves a bit
hsData = hsData[hsData["GrLivArea"] < 3500]
hsData = hsData[hsData["GarageArea"] < 1000]
hsData = hsData[hsData["Total_FlrSF"] < 3500]
hsData = hsData[hsData["Total_FlrSF"] > 500]
hsData = hsData[hsData["YearBuilt"] > 1888]
#hsData.info()
#hsData.describe()

In [ ]:
num_train_features = hsData.select_dtypes(include=[np.number])
num_train_features.dtypes

num_train_features = num_train_features.drop(['Id', 'MasVnrArea', 'GarageYrBlt', 'KitchenAbvGr','EnclosedPorch'], axis=1)
num_train_features['Neighborhood'] = hsData['Neighborhood'].apply(getNeighborMedian)

corr = num_train_features.corr()
print (corr['SalePrice'].sort_values(ascending=False)[:70], '\n')

# Drop 'Id', 'SalePrice' after evaluation
num_train_features = num_train_features.drop(['SalePrice'], axis=1)

#num_train_features.info()

In [ ]:
hsTest = pd.read_csv(test_file, header = 0)
hsTest = hsTest.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'LotFrontage', 'FireplaceQu', 'Utilities'], axis=1)

hsTest['Total_FlrSF'] = hsTest['1stFlrSF'] + hsTest['2ndFlrSF']

hsTest['FullBath'] = hsTest['FullBath'].fillna(0)
hsTest['BsmtFullBath'] = hsTest['BsmtFullBath'].fillna(0)
hsTest['HalfBath'] = hsTest['HalfBath'].fillna(0)
hsTest['BsmtHalfBath'] = hsTest['BsmtHalfBath'].fillna(0)
hsTest['Total_Bath'] = hsTest['FullBath'] + hsTest['BsmtFullBath'] + hsTest['HalfBath']*0.5 + hsTest['BsmtHalfBath']*0.5

hsTest['GarageQual'] = hsTest['GarageQual'].fillna('NA')
hsTest['GarageCond'] = hsTest['GarageCond'].fillna('NA')
hsTest['Garage_Composite'] = hsTest['GarageQual'] + hsTest['GarageCond']
hsTest['Garage_Composite'].replace(['PoPo','PoFa','NANA','FaFa','NAFa','FaPo','FaTA','ExEx','TAFa','TAPo','TAGd', \
                             'TATA','NATA','GdTA','GdGd','GdEx','ExTA'], \
                             [97,100,100,110,110,116,116,125,125,110,144,171,171,209,227,250,475], inplace=True)

hsTest['ExterQual'] = hsTest['ExterQual'].fillna('NA')
hsTest['ExterCond'] = hsTest['ExterCond'].fillna('NA')
hsTest['Exter_Composite'] = hsTest['ExterQual'] + hsTest['ExterCond']
hsTest['Exter_Composite'].replace(['FaPo','FaFa','FaGd','TAPo','FaTA','TAFa','TAEx','TATA','TAGd','GdGd', \
                             'GdFa','GdTA','GdEx','ExGd', 'ExEx', 'ExTA'], \
                             [50,61,70,77,99,119,140,140,140,210,210,221,230,267,325,374], inplace=True)

hsTest['BsmtQual'] = hsTest['BsmtQual'].fillna('NA')
hsTest['BsmtCond'] = hsTest['BsmtCond'].fillna('NA')
hsTest['Bsmt_Composite'] = hsTest['BsmtQual'] + hsTest['BsmtCond']
hsTest['Bsmt_Composite'].replace(['FaPo','FaGd','NANA','FaTA','FaFa','NAFa','TAPo','TAFa','TATA','TANA','NATA','TAGd','GdFa', \
                            'GdTA','GdGd','GdNA','ExPo','ExGd','ExTA'], \
                             [64,80,112,114,114,114,114,114,135,135,135,164,182,192,199,199,260,275,318], inplace=True)

hsTest['Year_Composite'] = hsTest['YearBuilt'] + hsTest['YearRemodAdd']
hsTest['Year_Composite'] = hsTest['Year_Composite'].apply(lambda year: year - 3700)

hsTest['HeatingQC'].replace(['Po','Fa','TA','Gd','Ex'], [87,124,135,152,195], inplace=True)

hsTest['KitchenQual'] = hsTest['KitchenQual'].fillna('Po')
hsTest['KitchenQual'].replace(['Po','Fa','TA','Gd','Ex'], [105565,115000,137000,201400,316750], inplace=True)

hsTest['Condition1'].replace(['Artery', 'Feedr', 'RRAe', 'Norm', 'RRAn', 'RRNe', 'PosN', 'PosA', 'RRNn'], [120,140,143,167,171,191,200,212,214], inplace=True)
hsTest['Condition2'].replace(['RRNn', 'Artery', 'Feedr', 'RRAn', 'Norm', 'RRAe', 'PosN', 'PosA'], [97,107,128,137,164,190,285,325], inplace=True)

hsTest['CentralAir'] = hsTest['CentralAir'].apply(lambda air: 1 if air == 'Y' else 0)

hsTest['Electrical'].replace(['Mix', 'FuseP', 'FuseF', 'FuseA', 'SBrkr'], [67,82,115,121,170], inplace=True)

hsTest['Functional'] = hsTest['Functional'].fillna('Min1')
hsTest['Functional'].replace(['Maj2', 'Sev', 'Mod', 'Min1', 'Min2', 'Maj1', 'Typ'], [85,129,140,140,140,140,166], inplace=True)

hsTest['SaleType'] = hsTest['SaleType'].fillna('ConLw')
hsTest['SaleType'].replace(['Oth', 'ConLI', 'COD', 'ConLD', 'ConLw', 'WD', 'CWD', 'New', 'Con'], [116,125,140,140,144,158,189,247,270], inplace=True)

hsTest['SaleCondition'].replace(['AdjLand', 'Abnorml', 'Family', 'Alloca', 'Normal', 'Partial'], [104,130,141,148,160,245], inplace=True)

hsTest['GarageType'] = hsTest['GarageType'].fillna('Basment')
hsTest['GarageType'].replace(['CarPort', 'Detchd', 'Basment', '2Types', 'Attchd', 'BuiltIn'], [108,130,148,159,185,228], inplace=True)

hsTest['MSZoning'] = hsTest['MSZoning'].fillna('RH')
hsTest['MSZoning'].replace(['C (all)','RM','RH','RL','FV'], [75,121,137,174,206], inplace=True)

hsTest['MSSubClass'] = hsTest['MSSubClass'].apply(str)
hsTest['MSSubClass'].replace(['180','30','45','190','50','90','85','40','160','70','20','75','80','120','150','60'], [89,100,108,128,132,136,141,143,146,156,159,164,167,192,192,215], inplace=True)

hsTest['BldgType'].replace(['2fmCon','Duplex','Twnhs','1Fam','TwnhsE'], [128,137,137,168,172], inplace=True)

hsTest['HouseStyle'].replace(['1.5Unf','1.5Fin','2.5Unf','SFoyer','1Story','SLvl','2Story','2.5Fin'], [111,132,135,135,155,165,190,194], inplace=True)

hsTest['LotShape'].replace(['Reg','IR1','IR2','IR3'], [146,189,204,221], inplace=True)

hsTest['LandContour'].replace(['Lvl','Bnk','Low','HLS'], [139,163,190,222], inplace=True)

hsTest['GarageFinish'] = hsTest['GarageFinish'].fillna('Unf')
hsTest['GarageFinish'].replace(['Unf','RFn','Fin'], [135,190,215], inplace=True)

print hsTest['KitchenQual'].unique()

#hsTest.info()
#hsTest.describe()

In [ ]:
num_test_features = hsTest.select_dtypes(include=[np.number])
num_test_features.dtypes

num_test_features = num_test_features.drop(['Id', 'MasVnrArea', 'GarageYrBlt', 'KitchenAbvGr','EnclosedPorch'], axis=1)

num_test_features['GarageCars'] = num_test_features['GarageCars'].fillna(0)
num_test_features['GarageArea'] = num_test_features['GarageArea'].fillna(0)
num_test_features['BsmtFinSF1'] = num_test_features['BsmtFinSF1'].fillna(0)
num_test_features['BsmtFinSF2'] = num_test_features['BsmtFinSF2'].fillna(0)
num_test_features['BsmtUnfSF'] = num_test_features['BsmtUnfSF'].fillna(0)
num_test_features['TotalBsmtSF'] = num_test_features['TotalBsmtSF'].fillna(0)
num_test_features['BsmtFullBath'] = num_test_features['BsmtFullBath'].fillna(0)
num_test_features['BsmtHalfBath'] = num_test_features['BsmtHalfBath'].fillna(0)
num_test_features['Neighborhood'] = hsTest['Neighborhood'].apply(getNeighborMedian)

#num_test_features.info()
num_test_features.isnull().values.any()

null_columns=num_test_features.columns[num_test_features.isnull().any()]
num_test_features[null_columns].isnull().sum()


In [ ]:
# Lasso Regressor
from sklearn.linear_model import Lasso
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

ls = Lasso(alpha=0.00055)
ls.fit(X_train, y_train)

X_test_predict = ls.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(ls)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Hs_Test = num_test_features
Hs_test_predict = ls.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Lasso Regressor')
plt.show()

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict) + 0.04705
print("Kaggle Possible: %s\n" % kg)

submit_file = 'c:\\house\\submission_ls_' + str(int(kg/0.000001)) + '.csv'
print submit_file
submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": Hs_test_predict
    })

submission.to_csv(submit_file, index=False)

model_file = 'c:\\house\\ls_' + str(int(kg/0.000001)) + '.pkl'
print model_file
joblib.dump(ls, model_file) 

In [ ]:
# LassoCV Regressor
from sklearn.linear_model import LassoCV
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

lscv = LassoCV(eps=10**-7, n_alphas=75)
lscv.fit(X_train, y_train)

X_test_predict = lscv.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(lscv)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Hs_Test = num_test_features
Hs_test_predict = lscv.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Lasso Regressor')
plt.show()

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict) + 0.04705
print("Kaggle Possible: %s\n" % kg)

submit_file = 'c:\\house\\submission_lscv_' + str(int(kg/0.000001)) + '.csv'
print submit_file
submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": Hs_test_predict
    })

submission.to_csv(submit_file, index=False)

model_file = 'c:\\house\\lscv_' + str(int(kg/0.000001)) + '.pkl'
print model_file
joblib.dump(ls, model_file) 

In [ ]:
# XG boost, by 'conda install py-xgboost'
from xgboost import XGBRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

xg = XGBRegressor(max_depth=5, n_estimators=700)

xg.fit(X_train, y_train)

X_test_predict = xg.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(xg)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Hs_Test = num_test_features
Hs_test_predict = xg.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('XG Boost')
plt.show()

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict) + 0.04705
print("Kaggle Possible: %s\n" % kg)

submit_file = 'c:\\house\\submission_xg_' + str(int(kg/0.000001)) + '.csv'
print submit_file
submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": Hs_test_predict
    })

submission.to_csv(submit_file, index=False)

model_file = 'c:\\house\\xg_' + str(int(kg/0.000001)) + '.pkl'
print model_file
joblib.dump(xg, model_file) 

In [ ]:
# Bagging Regressor
from sklearn.ensemble import BaggingRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

bg = BaggingRegressor(n_estimators=15)
bg.fit(X_train, y_train)

X_test_predict = bg.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(bg)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Bagging Regressor')
plt.show()

In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

rf = RandomForestRegressor(n_estimators = 150, max_depth=5)
rf.fit(X_train, y_train)

X_test_predict = rf.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(rf)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Random Forest Regressor')
plt.show()

In [ ]:
# Epsilon-Support Vector Regressor
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

svr = SVR(gamma=0.001, C=100, kernel='rbf')
svr.fit(X_train, y_train)

X_test_predict = svr.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(svr)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('SVR Regressor')
plt.show()


In [ ]:
from sklearn.externals import joblib

loadLs = joblib.load('c:\\house\\ls_118820.pkl') 

Hs_Test = num_test_features
Hs_test_predict = loadLs.predict(Hs_Test)
Hs_test_predict = np.exp(Hs_test_predict)

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

kg = rmsle(alpSlp, Hs_test_predict)+ 0.04705
print("Kaggle Possible: %s\n" % kg)
